# E-Commerce Analysis Project
This project demonstrates an end-to-end data analytics workflow for analyzing e-commerce transactions. The analysis aims to support business decision-making by identifying top-selling products, understanding monthly revenue trends, and uncovering customer purchasing patterns.

The dataset consists of approximately 250,000 UK-based online retail transactions. Each record contains the following eight attributes:
- InvoiceNo
- StockCode
- Description
- Quantity
- InvoiceDate
- UnitPrice
- CustomerID
- Country


## Data Cleaning

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("uk_ecommerce.csv", encoding='ISO-8859-1')

# Initial exploration
print(df.head())
print(df.info())
print(df.isnull().sum())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo   

In [5]:
df = df.dropna(subset=['CustomerID'])

In [6]:
df = df.drop_duplicates()

In [7]:
unwanted = 'discount|manual|post|bank charges|D|C2|M|DOT'
df = df[~df['Description'].str.contains(unwanted, case=False, na=False)]

In [8]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True, errors='coerce')

In the data cleaning step, transactions without a CustomerID were removed, duplicate records were eliminated, non-product entries (such as bank charges, postage, and manual entries) were filtered out, and the date format was converted to a DATE type compatible with SQL.

In [13]:
# Save Clean Data for SQL & Tableau
customers = df[['CustomerID', 'Country']].drop_duplicates()
customers.to_csv('customers_clean.csv', index=False)

products = df[['StockCode', 'Description', 'UnitPrice']].drop_duplicates()
products.to_csv('products_clean.csv', index=False)

orders = df[['InvoiceNo', 'CustomerID', 'StockCode', 'Quantity', 'InvoiceDate']]
orders.to_csv('orders_clean.csv', index=False)

The cleaned dataset was exported to MySQL for analytical querying and subsequently used for visualization in Tableau.

# Analysis using SQL Quiries

The following SQL query calculates total customer spending based on
order quantity and unit price.

```sql
SELECT o.customer_id, SUM(o.quantity * p.unit_price)
AS total_spent
FROM orders o
JOIN products p ON o.stock_code = p.stock_code
GROUP BY o.customer_id
ORDER BY total_spent DESC
LIMIT 10;
```
![Top 10 Customers with Most Total Spent](customer.png)


The following SQL query calculates total revenue for top 10 products based on order quantity and unit price.

```sql
SELECT p.description, SUM(o.quantity * p.unit_price) AS revenue
FROM orders o
JOIN products p ON o.stock_code = p.stock_code
GROUP BY p.description
ORDER BY revenue DESC
LIMIT 10;
```
![10 Products with the Most Revenue](products.png)


The following SQL query returned the products with the most total returned / cancel purchase.

```sql
SELECT p.description, SUM(ABS(o.quantity)) AS total_returned
FROM orders o
JOIN products p ON o.stock_code = p.stock_code
WHERE o.quantity < 0
GROUP BY p.description
ORDER BY total_returned DESC
LIMIT 10;
```
![10 Most Returned Products](returned.png)


The following SQL query showed the total revenue for each month.
```sql
SELECT DATE_FORMAT(invoice_date, '%Y-%m') AS month, 
       SUM(quantity * p.unit_price) AS monthly_revenue
FROM orders o
JOIN products p ON o.stock_code = p.stock_code
GROUP BY month
ORDER BY month;
```
![Monthly Revenue Trend](trend.png)


The following SQL query showed the 10 country with the most total revenue.
```sql
SELECT c.country, SUM(o.quantity * p.unit_price) AS total_revenue
FROM orders o
JOIN products p ON o.stock_code = p.stock_code
JOIN customers c ON o.customer_id = c.customer_id
GROUP BY c.country
ORDER BY total_revenue DESC;
```
![Top 10 Countries with the Most Revenue](country.png)

# Data Visualization using Tableau
The final insights are presented in an interactive Tableau dashboard.

Key visuals:
- Top 10 Customers by Revenue
- Monthly Revenue Trend (Line Chart)
- Top 10 Products by Revenue
- Most Returned Products
- Revenue by Country

🔗 Tableau Public Link: https://prod-ca-a.online.tableau.com/#/site/firilanjm-c5ba934292/views/E-CommerceAnalysis/Dashboard1?:iid=2 





![Top Customers](ecommerce.png)

## Key Insights & Business Implications

Revenue is highly concentrated among a single top customer, indicating potential business risk if this customer becomes inactive. To create amore stable revenue, the company should focus on engaging mid-tier customers through targeted promotions or loyalty programs.

From October to December 2011, a slight decline of total revenue suggests possible seasonal effects, increased returns, or reduced demand. Further analysis is needed to identify the root cause and prevent similar declines in the future.

Some products experience exceptionally high return volumes, with one product exceeding 1400 returned items. This may point to quality issues, non-obvious descriptions, or customer dissatisfaction and should be investigated to reduce losses. 

Finally, most revenue comes from the United Kingdom, which is expected given the company's base. However, expanding marketing and ales efforts to other countries could help diversify revenue and support long-term growth.

## Conclusion

This project demonstrates an e-commerce dataset using SQL for data aggregation, Python for data preparation, and Tableau for visualization to identify revenue patterns, customer behavior, and return trends. The analysis revealed revenue concentration among a small number of customers, a slight decline in late-2011 revenue, high return volumes for certain products, and heavy dependence on the UK market. These findings highlight opportunities to improve customer retention, reduce returnd, and expand internationally. 